In [1]:
import subprocess
import re
import random
import sqlite3
import io
import csv

import locations
import parse_cregit
import csv_to_sql, sql_queries

In [2]:
def all_tags():
    result = subprocess.run("git tag", cwd=locations.tokenized_kernel, capture_output=True, shell=True)
    return str(result.stdout, 'utf-8').strip().split("\n")

def tags():
    result = []
    for tag in all_tags():
        m = re.match(r'^v(\d+)\.(\d+)$', tag)
        if m:
            major, minor = m.groups()
            result.append((tag, int(major), int(minor)))
    return result

In [ ]:
for tag, major, minor in tags()[2:]:
    subprocess.run(f"git checkout {tag}", cwd=locations.tokenized_kernel, shell=True, check=True)
    outdir = locations.by_tag/tag
    outdir.mkdir(exist_ok=True)
    parse_cregit.parse_all(outdir)
#     parse_cregit.parse(
#         random.sample(list(locations.tokenized_kernel.rglob("*.c")), 20),
#         outdir                       
#     )
    
    connection = sqlite3.connect(outdir/"function_survey.db")
    csv_to_sql.reset_all(source_dir=outdir, connection=connection)
    sql_queries.make_paths_table(root=locations.tokenized_kernel, connection=connection)
    sql_queries.make_functions_view(connection)
    result = sql_queries.aggregate_coverage(
        connection,
        sql_queries.TESTED_IDENTIFIERS,
        sql_queries.NON_STATIC,
        max_levels = 2
    )
    
    buffer = io.StringIO()
    writer = csv.writer(buffer)
    for x in result:
        print
        writer.writerow(x)
    with open(outdir/"coverage_by_directory.csv", "w") as f:
        f.write(buffer.getvalue())